# Regression Model B: Field 2

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 2 (F2)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E2" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E2" ),]
dE4=d4[which(d4$Estate=="E2" ),]
dE5=d5[which(d5$Estate=="E2" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
13,1,1,1,1,1,1,1,1,1,1,1,0
15,1,1,1,1,1,1,1,1,1,0,0,2
4,1,1,1,1,1,1,1,0,0,1,1,2
1,1,1,1,1,1,0,0,1,1,1,1,2
7,1,1,1,1,1,1,1,0,0,0,0,4
3,1,1,1,1,1,0,0,1,1,0,0,4
1,1,1,1,1,1,0,0,0,0,1,1,4
,0,0,0,0,0,5,5,12,12,25,25,84


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   3  prun.time  prun.cost  manuring.time

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            25            25            12            12 
      pd.time       pd.cost 
            5             5 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost       pd.time 
            2             3             4             5             6 
      pd.cost 
            7 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1           

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper")

## Train and test data

Split the dataset into train and test sets. After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

In this case, the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 


In [10]:
train = df2[df2$year<=2016,]
test = df2[df2$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [11]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-53.702 -24.251  -0.562  21.045  63.182 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)  
(Intercept)   44.2060881 32.8496996   1.346   0.1915  
rainfall       0.0363681  0.0319219   1.139   0.2663  
foliar        -3.9847583  2.2656280  -1.759   0.0919 .
prun.time      4.2184512  5.7044166   0.740   0.4671  
prun.cost      0.0091920  0.0070142   1.310   0.2030  
manuring.time 27.8929918 18.4097220   1.515   0.1434  
manuring.cost  0.0002387  0.0006422   0.372   0.7135  
pd.time       23.2412440 11.1097956   2.092   0.0477 *
pd.cost       -0.0080700  0.0072472  -1.114   0.2770  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 35.34 on 23 degrees of freedom
Multiple R-squared:  0.3844,	Adjusted R-squared:  0.1703 
F-statistic: 1.79

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [12]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,3718.0460,3718.0460,2.9767570,0.09788186
foliar,1,1519.0578,1519.0578,1.2161942,0.28151482
prun.time,1,1322.0634,1322.0634,1.0584757,0.31426174
prun.cost,1,1196.1512,1196.1512,0.9576674,0.33795721
manuring.time,1,4530.1539,4530.1539,3.6269501,0.06943309
manuring.cost,1,190.1464,190.1464,0.1522358,0.69999792
pd.time,1,3917.8005,3917.8005,3.1366852,0.08980210
pd.cost,1,1548.7493,1548.7493,1.2399659,0.27698005
Residuals,23,28727.5913,1249.0257,NA,NA


### Using stepwise

In [13]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=235.6
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.cost  1     172.5 28900 233.79
- prun.time      1     683.1 29411 234.35
- pd.cost        1    1548.7 30276 235.28
- rainfall       1    1621.2 30349 235.35
<none>                       28728 235.60
- prun.cost      1    2145.0 30873 235.90
- manuring.time  1    2867.3 31595 236.64
- foliar         1    3863.7 32591 237.63
- pd.time        1    5466.1 34194 239.17

Step:  AIC=233.79
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- prun.time      1     584.0 29484 232.43
- pd.cost        1    1801.8 30702 233.72
<none>                       28900 233.79
- prun.cost      1    1977.1 30877 233.91
- rainfall       1    2482.1 31382 234.42
- manuring.time  1    3139.6 32040 235.09
- foliar         1    3696.9 32597 235.64
- pd.ti


Call:
lm(formula = yield ~ rainfall + foliar + pd.time, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-50.252 -25.549  -1.428  16.160  73.606 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept) 79.75636   25.76384   3.096  0.00443 **
rainfall     0.05766    0.02814   2.049  0.04994 * 
foliar      -2.76675    1.91974  -1.441  0.16062   
pd.time     20.35419    9.12635   2.230  0.03392 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 35.45 on 28 degrees of freedom
Multiple R-squared:  0.2461,	Adjusted R-squared:  0.1654 
F-statistic: 3.047 on 3 and 28 DF,  p-value: 0.04505


rainfall   foliar  pd.time 
1.049673 1.032422 1.075082 



Call:
lm(formula = yield ~ rainfall + foliar + pd.time, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-50.252 -25.549  -1.428  16.160  73.606 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept) 79.75636   25.76384   3.096  0.00443 **
rainfall     0.05766    0.02814   2.049  0.04994 * 
foliar      -2.76675    1.91974  -1.441  0.16062   
pd.time     20.35419    9.12635   2.230  0.03392 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 35.45 on 28 degrees of freedom
Multiple R-squared:  0.2461,	Adjusted R-squared:  0.1654 
F-statistic: 3.047 on 3 and 28 DF,  p-value: 0.04505


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [14]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,107.799,116.49322
34,61.336,121.85683
35,62.955,98.38993
36,55.218,137.65706
37,49.703,149.23691
38,75.186,123.11801
39,164.932,123.26307
40,256.238,116.14268
41,158.787,199.17451


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (  ) to know how close the data to the fitted regression line. 

In [15]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(k1)$r.squared)
fit

RMSE         R2 
67.6758057  0.3844496